In [13]:
from collections import Counter
from itertools import product

In [14]:
with open('../data/Human_BRCA2_orthologues2.fa') as file: righe = file.readlines()

In [15]:
import re
geni = []
for riga in righe:
    riga = riga.rstrip('\n')
    if re.match('^>', riga):
        nuovo_gene = {'sequence': ''}
        geni.append(nuovo_gene)
        nuovo_gene['header'] = re.sub('^>', '', riga)
    elif not re.match('^;', riga):
        nuovo_gene['sequence'] = nuovo_gene['sequence'] + riga

In [16]:
lunghezza = 10
sequenze = [gene['sequence'] for gene in geni][0:5]
num_stringhe = len(sequenze)

In [17]:
def moda(lista):
    c = ''
    conta = Counter(lista)
    val = max(conta.values())
    for c in conta.keys():
        if conta[c] == val:  
            return {'carattere': c, 'valore': val}

Una funzione che riceve un insieme di stringhe e calcola il suo consenso, restituendo un dizionario con chiavi ```stringa``` e ```valore```.

In [18]:
def consenso(stringhe):
    colonne = list(zip(*stringhe))
    caratteri = ''.join([moda(col)['carattere'] for col in colonne])
    valore = sum([moda(col)['valore'] for col in colonne])
    return((caratteri, valore))

In [19]:
def sottostringhe(stringhe, inizio):
     return [x[0][x[1]: x[1] + lunghezza] for x in zip(stringhe,inizio)]

In [51]:
def prossimi(prec, indice, massimi):
    for livello in range(indice, -1, -1):
        prec[livello] += 1
        if prec[livello] < massimi[livello]:
            for i in range(livello + 1, lunghezza):
                prec[i] = 0
            return prec
    return None

Calcola l'array dei punti di inizio, data la stringa di consenso

In [53]:
def uguali(s, t):
    return len([p for p in zip(s,t) if p[0] == p[1]])

In [54]:
print(uguali('ACC', 'AGC'))
print(uguali('ACC', 'ACC'))
print(uguali('ACC', 'CCC'))
print(uguali('CCT', 'CTG'))

2
3
2
1


In [55]:
def uguali2(stringhe, t):
    return sum([uguali(s,t) for s in stringhe])

In [58]:
def punto(stringa, consenso):
    l = len(consenso)
    ok = [uguali(stringa[i : i + l], consenso) for i in range(len(stringa) - l +1)]
    return ok.index(max(ok))

In [59]:
print(punto('ACGACGACGACT', 'GGAG'))
print(punto('ACGACTACGTAC', 'GGAG'))
print(punto('ACGGACTACGAC', 'GGAG'))

1
1
2


In [26]:
def punti(stringhe, consenso):
    return [punto(s, consenso) for s in stringhe]

Trasforma un insieme di numeri fra 0 e 3 in una stringa sull'alfabeto ACGT.

In [27]:
def simboli2stringa(simboli):
    alfabeto = 'ACGT'
    return ''.join([ alfabeto[x] for x in simboli])

In [28]:
print(simboli2stringa([0,1,0,2]))

ACAG


Calcolo il valore massimo teorico di ogni soluzione dove le prime ```quante``` stringhe sono fissate.

In [65]:
def branch_and_bound():
    simboli = [0] * lunghezza
    massimi = [4] * lunghezza
    motivo = ''
    max_valore = 0
    while(simboli != None):
        soluzione = simboli2stringa(simboli)
        punti_inizio = punti(sequenze, soluzione)
#        print("  " + soluzione)
        estratte = sottostringhe(sequenze, punti_inizio)
        valore = uguali2(estratte, soluzione)
        
        if valore > max_valore:
            max_valore = valore
            motivo = soluzione
            print(soluzione, valore, punti_inizio)
        
        if max_valore == num_stringhe * lunghezza:
            return (motivo, max_valore)

        for buone in range(lunghezza - 1, -1, -1):
            if uguali2(estratte, soluzione[:buone]) + num_stringhe * (lunghezza - buone) > valore:
                break
        
        simboli = prossimi(simboli, buone, massimi)
            
    return (motivo, max_valore)

In [66]:
%time print(branch_and_bound())

AAAAAAAAAA 45 [596, 782, 633, 35, 1660]
AAGATGTAAT 46 [853, 824, 37, 220, 934]
CTGGAGATCT 47 [940, 265, 319, 307, 1021]
('CTGGAGATCT', 47)
CPU times: user 41min 24s, sys: 476 ms, total: 41min 24s
Wall time: 41min 24s
